# MIMIC mortality (unsupervised)

This notebook reproduces the unsupervised SUAVE mortality analysis.

In [1]:

import sys
import json
from pathlib import Path
import time
from typing import Dict, List, Mapping, Optional, Tuple
from IPython.display import Markdown, display

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

EXAMPLES_DIR = Path().resolve()
if not EXAMPLES_DIR.exists():
    raise RuntimeError("Run this notebook from the repository root so 'examples' is available.")
if str(EXAMPLES_DIR) not in sys.path:
    sys.path.insert(0, str(EXAMPLES_DIR))

from mimic_mortality_utils import (
    RANDOM_STATE,
    TARGET_COLUMNS,
    CALIBRATION_SIZE,
    VALIDATION_SIZE,
    Schema,
    define_schema,
    compute_auc,
    format_float,
    load_dataset,
    prepare_features,
    schema_markdown_table,
    split_train_validation_calibration,
    to_numeric_frame,
)

from suave import SUAVE
from suave.evaluate import (
    evaluate_tstr,
    evaluate_trtr,
    simple_membership_inference,
    kolmogorov_smirnov_statistic,
    mutual_information_feature,
    rbf_mmd,
)


In [2]:
# Configuration
DATA_DIR = (EXAMPLES_DIR / "data" / "sepsis_mortality_dataset").resolve()
OUTPUT_DIR = EXAMPLES_DIR / "analysis_outputs_unsupervised"
OUTPUT_DIR.mkdir(exist_ok=True)

train_df = load_dataset(DATA_DIR / "mimic-mortality-train.tsv")
test_df = load_dataset(DATA_DIR / "mimic-mortality-test.tsv")
external_df = load_dataset(DATA_DIR / "eicu-mortality-external_val.tsv")

FEATURE_COLUMNS = [column for column in train_df.columns if column not in [*TARGET_COLUMNS, 'PaO2']]
schema = define_schema(train_df, FEATURE_COLUMNS)

# manual schema correction
schema.update({'BMI':{'type': 'real'}})
schema.update({'Respiratory_Support':{'type': 'ordinal', 'n_classes': 5}})
schema.update({'LYM%':{'type': 'real'}})

schema_table = schema_markdown_table(schema)
display(Markdown(schema_table))

[schema] Column 'age' flagged for review: Integer feature near categorical threshold.
[schema] Column 'PaO2/FiO2' flagged for review: Positive skew close to threshold.


| Column | Type | n_classes | y_dim |
| --- | --- | --- | --- |
| age | real |  |  |
| sex | cat | 2 |  |
| BMI | real |  |  |
| temperature | real |  |  |
| heart_rate | real |  |  |
| respir_rate | real |  |  |
| SBP | real |  |  |
| DBP | real |  |  |
| MAP | real |  |  |
| SOFA_cns | ordinal | 5 |  |
| CRRT | cat | 2 |  |
| Respiratory_Support | ordinal | 5 |  |
| WBC | pos |  |  |
| Hb | real |  |  |
| NE% | real |  |  |
| LYM% | real |  |  |
| PLT | pos |  |  |
| ALT | pos |  |  |
| AST | pos |  |  |
| STB | pos |  |  |
| BUN | pos |  |  |
| Scr | pos |  |  |
| Glu | pos |  |  |
| K+ | real |  |  |
| Na+ | real |  |  |
| Fg | pos |  |  |
| PT | pos |  |  |
| APTT | pos |  |  |
| PH | real |  |  |
| PaO2/FiO2 | pos |  |  |
| PaCO2 | pos |  |  |
| HCO3- | real |  |  |
| Lac | pos |  |  |

In [3]:

def make_latent_classifier() -> Pipeline:
    """Return the logistic regression pipeline used on latent representations."""

    return Pipeline(
        [
            ("scaler", StandardScaler()),
            ("classifier", LogisticRegression(max_iter=1000)),
        ]
    )


def make_logistic_pipeline() -> Pipeline:
    """Factory for the baseline classifier used in TSTR/TRTR."""

    return Pipeline(
        [
            ("scaler", StandardScaler()),
            ("classifier", LogisticRegression(max_iter=200)),
        ]
    )

In [4]:

metrics_records: List[Dict[str, object]] = []
membership_records: List[Dict[str, object]] = []

latent_models: Dict[str, Pipeline] = {}
suave_models: Dict[str, SUAVE] = {}

tstr_results: Optional[pd.DataFrame] = None
tstr_path: Optional[Path] = None
distribution_df: Optional[pd.DataFrame] = None
distribution_path: Optional[Path] = None

for target in TARGET_COLUMNS:
    if target not in train_df.columns:
        continue
    print(f"Training unsupervised model for {target}…")
    X_full = prepare_features(train_df, FEATURE_COLUMNS)
    y_full = train_df[target]

    (
        X_train_model,
        X_validation,
        y_train_model,
        y_validation,
    ) = train_test_split(
        X_full,
        y_full,
        test_size=VALIDATION_SIZE,
        random_state=RANDOM_STATE,
        stratify=y_full
    )

    hidden_dimension_options: Dict[str, Tuple[int, int]] = {
        "compact": (128, 64),
        "balanced": (256, 128),
        "widened": (384, 192),
        "extended": (512, 256),
    }
    model = SUAVE(
        schema=schema,
        behaviour="unsupervised",
        latent_dim=32,
        hidden_dims=(384, 192),
        dropout=0.01,
        learning_rate=0.0003,
        batch_size=1024,
        beta=1.0,
        n_components=7,
        tau_start=4,
        tau_min=0.04,
        tau_decay=0.001,
        random_state=RANDOM_STATE,
    )
    model.fit(
        X_train_model,
        warmup_epochs=50,
        kl_warmup_epochs=6,
        plot_monitor=True
    )
    suave_models[target] = model

    latent_classifier = make_latent_classifier()
    train_latents = model.encode(X_train_model)

    evaluation_datasets: Dict[str, Tuple[pd.DataFrame, pd.Series]] = {
        "Train": (X_train_model, y_train_model),
        "Validation": (X_validation, y_validation),
        "MIMIC test": (
            prepare_features(test_df, FEATURE_COLUMNS),
            test_df[target],
        ),
    }
    if target in external_df.columns:
        evaluation_datasets["eICU external"] = (
            prepare_features(external_df, FEATURE_COLUMNS),
            external_df[target],
        )

    latent_classifier.fit(train_latents, np.asarray(y_train_model))
    latent_models[target] = latent_classifier

    for dataset_name, (features, labels) in evaluation_datasets.items():
        latents = model.encode(features)
        probs = latent_classifier.predict_proba(latents)
        auc = compute_auc(probs, labels)
        metrics_records.append(
            {
                "target": target,
                "dataset": dataset_name,
                "auc": auc,
            }
        )

    train_probs = latent_classifier.predict_proba(train_latents)
    test_latents = model.encode(evaluation_datasets["MIMIC test"][0])
    test_probs = latent_classifier.predict_proba(test_latents)
    membership = simple_membership_inference(
        train_probs,
        np.asarray(y_train_model),
        test_probs,
        np.asarray(evaluation_datasets["MIMIC test"][1]),
    )
    membership_records.append({"target": target, **membership})

metrics_df = pd.DataFrame(metrics_records)
metrics_path = OUTPUT_DIR / "evaluation_metrics_unsupervised.csv"
metrics_df.to_csv(metrics_path, index=False)

membership_df = pd.DataFrame(membership_records)
membership_path = OUTPUT_DIR / "membership_inference_unsupervised.csv"
membership_df.to_csv(membership_path, index=False)

primary_target = "in_hospital_mortality"
if primary_target in suave_models and primary_target in latent_models:
    print("Generating synthetic data for TSTR/TRTR comparisons…")
    model = suave_models[primary_target]
    latent_classifier = latent_models[primary_target]

    X_train_full = prepare_features(train_df, FEATURE_COLUMNS)
    y_train_full = train_df[primary_target]
    numeric_train = to_numeric_frame(X_train_full)
    train_means = numeric_train.mean(axis=0)
    train_means = train_means.fillna(0.0)
    numeric_train = numeric_train.fillna(train_means)

    synthetic_features = model.sample(len(X_train_full))
    synthetic_features = synthetic_features[FEATURE_COLUMNS]
    numeric_synthetic = to_numeric_frame(synthetic_features)
    numeric_synthetic = numeric_synthetic.fillna(train_means)

    synthetic_latents = model.encode(synthetic_features)
    synthetic_probs = latent_classifier.predict_proba(synthetic_latents)[:, 1]
    rng = np.random.default_rng(RANDOM_STATE)
    synthetic_labels = rng.binomial(1, synthetic_probs)

    numeric_test = to_numeric_frame(prepare_features(test_df, FEATURE_COLUMNS))
    numeric_test = numeric_test.fillna(train_means)
    y_test = test_df[primary_target]

    tstr_metrics = evaluate_tstr(
        (numeric_synthetic.to_numpy(), synthetic_labels),
        (numeric_test.to_numpy(), y_test.to_numpy()),
        make_logistic_pipeline,
    )
    trtr_metrics = evaluate_trtr(
        (numeric_train.to_numpy(), y_train_full.to_numpy()),
        (numeric_test.to_numpy(), y_test.to_numpy()),
        make_logistic_pipeline,
    )
    tstr_results = pd.DataFrame(
        [
            {"setting": "TSTR", **tstr_metrics},
            {"setting": "TRTR", **trtr_metrics},
        ]
    )
    tstr_path = OUTPUT_DIR / "tstr_trtr_comparison_unsupervised.csv"
    tstr_results.to_csv(tstr_path, index=False)

    distribution_rows: List[Dict[str, object]] = []
    for column in FEATURE_COLUMNS:
        real_values = numeric_train[column].to_numpy()
        synthetic_values = numeric_synthetic[column].to_numpy()
        distribution_rows.append(
            {
                "feature": column,
                "ks": kolmogorov_smirnov_statistic(real_values, synthetic_values),
                "mmd": rbf_mmd(
                    real_values, synthetic_values, random_state=RANDOM_STATE
                ),
                "mutual_information": mutual_information_feature(
                    real_values, synthetic_values
                ),
            }
        )
    distribution_df = pd.DataFrame(distribution_rows)
    distribution_path = OUTPUT_DIR / "distribution_shift_metrics_unsupervised.csv"
    distribution_df.to_csv(distribution_path, index=False)
else:
    print("Primary target model not available; skipping TSTR/TRTR and distribution analysis.")


summary_lines: List[str] = [
    "# Unsupervised mortality modelling report",
    "",
    "## Schema",
    schema_table,
    "",
    "## Model selection and performance",
]


if tstr_results is not None:
    summary_lines.append("## TSTR vs TRTR")
    summary_lines.append("| Setting | Accuracy | AUC | AUPRC | Brier | ECE |")
    summary_lines.append("| --- | --- | --- | --- | --- | --- |")
    for _, row in tstr_results.iterrows():
        summary_lines.append(
            "| {setting} | {acc:.3f} | {auc:.3f} | {auprc:.3f} | {brier:.3f} | {ece:.3f} |".format(
                setting=row["setting"],
                acc=row.get("accuracy", np.nan),
                auc=row.get("auroc", np.nan),
                auprc=row.get("auprc", np.nan),
                brier=row.get("brier", np.nan),
                ece=row.get("ece", np.nan),
            )
        )
    summary_lines.append("")

summary_lines.append("## Distribution shift and privacy")
if distribution_df is not None and distribution_path is not None:
    distribution_top = distribution_df.sort_values("ks", ascending=False).head(10)
    summary_lines.append("Top 10 features by KS statistic:")
    summary_lines.append("| Feature | KS | MMD | Mutual information |")
    summary_lines.append("| --- | --- | --- | --- |")
    for _, row in distribution_top.iterrows():
        summary_lines.append(
            "| {feature} | {ks:.3f} | {mmd:.3f} | {mi:.3f} |".format(
                feature=row["feature"],
                ks=row.get("ks", np.nan),
                mmd=row.get("mmd", np.nan),
                mi=row.get("mutual_information", np.nan),
            )
        )
    summary_lines.append(
        f"Full distribution metrics: {distribution_path.relative_to(OUTPUT_DIR)}"
    )
else:
    summary_lines.append("Distribution metrics were not computed.")

if not membership_records:
    summary_lines.append("No membership inference metrics were recorded.")
else:
    summary_lines.append("Membership inference results:")
    summary_lines.append(
        "| Target | attack_auc | attack_accuracy | attack_threshold |"
    )
    summary_lines.append("| --- | --- | --- | --- |")
    for _, row in pd.DataFrame(membership_records).iterrows():
        summary_lines.append(
            "| {target} | {auc:.3f} | {accuracy:.3f} | {threshold:.3f} |".format(
                target=row["target"],
                auc=row.get("attack_auc", np.nan),
                accuracy=row.get("attack_best_accuracy", np.nan),
                threshold=row.get("attack_best_threshold", np.nan),
            )
        )
    summary_lines.append(
        f"Membership metrics saved to: {membership_path.relative_to(OUTPUT_DIR)}"
    )

summary_path = OUTPUT_DIR / "summary_unsupervised.md"
summary_path.write_text("\n".join(summary_lines), encoding="utf-8")

print("Analysis complete.")
print(f"Metric table saved to {metrics_path}")
print(f"Membership inference results saved to {membership_path}")
if tstr_path is not None and distribution_path is not None:
    print(f"TSTR/TRTR comparison saved to {tstr_path}")
    print(f"Distribution metrics saved to {distribution_path}")
print(f"Summary written to {summary_path}")


Training unsupervised model for in_hospital_mortality…


unsupervised training:   0%|          | 0/50 [00:00<?, ?it/s]

Training unsupervised model for 28d_mortality…


unsupervised training:   0%|          | 0/50 [00:00<?, ?it/s]

Generating synthetic data for TSTR/TRTR comparisons…
Analysis complete.
Metric table saved to E:\BaiduNetdiskWorkspace\Jupyter\my_repos\SUAVE\examples\analysis_outputs_unsupervised\evaluation_metrics_unsupervised.csv
Membership inference results saved to E:\BaiduNetdiskWorkspace\Jupyter\my_repos\SUAVE\examples\analysis_outputs_unsupervised\membership_inference_unsupervised.csv
TSTR/TRTR comparison saved to E:\BaiduNetdiskWorkspace\Jupyter\my_repos\SUAVE\examples\analysis_outputs_unsupervised\tstr_trtr_comparison_unsupervised.csv
Distribution metrics saved to E:\BaiduNetdiskWorkspace\Jupyter\my_repos\SUAVE\examples\analysis_outputs_unsupervised\distribution_shift_metrics_unsupervised.csv
Summary written to E:\BaiduNetdiskWorkspace\Jupyter\my_repos\SUAVE\examples\analysis_outputs_unsupervised\summary_unsupervised.md
